In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import random

TODO: look at this to add more measures
    https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python
        

In [2]:
G = nx.MultiDiGraph()
#df=pd.read_excel("./input/data.xlsx")
#df.info()
N=10
node_list=list(range(1,N+1))
rows=[]
T=100
t_list=range(1,T+1)
t_dict={}
for t in t_list:
    num_children=random.randint(1,10)
    start_node=random.randint(1,3)
    for e in range(num_children):
        j=random.randint(-1,2)
        days=round(random.random() * 10.0,1)
        next_node_index=min(N-1,start_node+j)
        next_node=node_list[next_node_index]
        if (next_node==N-1) | (next_node==start_node):
            continue
        G.add_edge(start_node, next_node, id=t, weight=days)
        start_node=next_node
 
        
    

In [3]:
for t in t_list:
    node_flow=[]
    for ori, end, data in G.edges(data=True):
        if  data['id']== t:
            node_flow.append([ori,end, data['weight']])
    total_days=sum([x[2] for x in node_flow])        
    t_dict[t]={}
    t_dict[t]['steps']=len(node_flow)
    t_dict[t]['days']=total_days
    

3 13.5


[(2, 4), (7, 8), (4, 7)]


https://networkx.org/documentation/stable/reference/generated/networkx.classes.function.subgraph_view.html

In [11]:
import functools
def filter_edge(n1, n2, key,id):
    if G[n1][n2][key].get("id")==id:
        if n1 != n2:
            return True
        else:
            return False
    else:
        return False

def filter_node(n, id):
    for n1,n2,data in G.in_edges(n, data=True):
        if data["id"]==id:
            return True
    for n1,n2,data in G.out_edges(n, data=True):
        if data["id"]==id:
            return True
    return False
    
t_process=[]
for i, t in enumerate(t_list):
    filter_edge_partial=functools.partial(filter_edge,id=t)
    filter_node_partial=functools.partial(filter_node,id=t)
    view = nx.subgraph_view(G , filter_node=filter_node_partial, filter_edge=filter_edge_partial)
    t_process.append(view)
    #print(i,"Edges: ",  view.edges(), "Nodes: ", view.nodes())

In [12]:
def find_root(G,child,n=0):
    parent = list(G.predecessors(child))
    if len(parent) == 0:
        #print(f"found root: {child}")
        return child
    else:  
        if n>100:
            return False
        else:
            return find_root(G, parent[0],n+1)
    

In [16]:
variant_dict={}
variant_sequence=[]
variant_counts=[]
for id in t_list:
    J=t_process[id-1]
    if len(J.nodes())>0:
        root_node=find_root(J,min(J.nodes()))
    else:
        continue
    if root_node:
        variant=list(nx.bfs_tree(J, root_node , reverse=False))
        try:
            i= variant_sequence.index(variant)
            variant_counts[i]+=1
        except:
            variant_sequence.append(variant)
            variant_counts.append(1)
            i=len(variant_counts)-1
    variant_dict[id]=i

In [17]:
print(len(variant_sequence))    
#print(variant_sequence)
print(variant_counts)

59
[2, 2, 1, 1, 4, 2, 1, 2, 4, 4, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 3, 1, 2, 1, 1, 3, 1, 2, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1]


In [26]:
variant_dict[98]

4

In [25]:
variant_sequence[20]

[3, 5, 8, 10]

In [ ]:
id=99
selected_edges = [(u,v) for u,v,e in G.edges(data=True) if e['id'] == id]
print (selected_edges)
print(t_dict[4]['steps'] , t_dict[4]['days'])
J=t_process[id]
print(J.nodes())
print(J.edges())
root_node=find_root(J,min(J.nodes()))
print("Root: ", root_node)
leaves= [x for x in J.nodes() if J.out_degree(x)==0 and J.in_degree(x)==1]
print("Leaves: ", leaves)
list(nx.bfs_tree(J, root_node , reverse=False))